# b. 使用大模型 API 对视频进行快速摘要（音频处理）- 精简版

这是精简的核心代码，非交互版，直接对应于文章内容，变量名相比于交互版会稍微修改，以更清晰的反映作用。

在这里，你可以一键执行直接看到结果。

如果你想更深入的体会其中的参数和代码细节，访问：[a. 交互完整版](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/13.%20轻松开始你的第一次%20AI%20视频总结（API%20版）.ipynb)

## 第 1 部分 - 准备工作


### 安装和导入

In [1]:
!pip install srt==3.5.3
!pip install datasets==2.20.0
!pip install DateTime==5.5
!pip install opencv-contrib-python==4.8.0.76
!pip install opencv-python==4.8.0.76
!pip install opencv-python-headless==4.10.0.84
!pip install openpyxl==3.1.4
!pip install openai==1.35.3
!pip install git+https://github.com/openai/whisper.git@ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
!pip install numpy==1.25.2
!pip install soundfile==0.12.1
!pip install librosa==librosa-0.10.2.post1

In [2]:
# 标准库
import os
import time
import re
import pathlib
import textwrap
import datetime

# 第三方库
import numpy as np
import srt
import soundfile as sf
from tqdm import tqdm

# 项目相关库
import whisper
from datasets import load_dataset
from openai import OpenAI

### 加载数据


In [3]:
# 加载本地 Parquet 格式的数据集
dataset = load_dataset('parquet', data_files={'test': './data/13/test-00000-of-00001.parquet'})

# 准备音频
input_audio = dataset["test"]["audio"][0]
input_audio_name = input_audio["path"]
input_audio_array = input_audio["array"].astype(np.float32)
sampling_rate = input_audio["sampling_rate"]

## 第 2 部分 - 自动语音识别 (ASR)

我们将使用 OpenAI 的 Whisper 模型将音频转换为字幕。

### 定义语音识别函数

In [4]:
def speech_recognition(model_name, input_audio, output_subtitle_path, decode_options, cache_dir="./"):
    # 加载模型
    model = whisper.load_model(name=model_name, download_root=cache_dir)

    # 转录音频
    transcription = model.transcribe(
        audio=input_audio,
        language=decode_options["language"],
        verbose=False,
        initial_prompt=decode_options["initial_prompt"],
        temperature=decode_options["temperature"]
    )

    # 处理转录结果，生成字幕文件
    subtitles = []
    for i, segment in enumerate(transcription["segments"]):
        start_time = datetime.timedelta(seconds=segment["start"])
        end_time = datetime.timedelta(seconds=segment["end"])
        text = segment["text"]
        subtitles.append(srt.Subtitle(index=i, start=start_time, end=end_time, content=text))

    srt_content = srt.compose(subtitles)

    # 保存字幕文件
    with open(output_subtitle_path, "w", encoding="utf-8") as file:
        file.write(srt_content)

    print(f"字幕已保存到 {output_subtitle_path}")

### 设置参数


In [5]:
# 模型名称，可选 'tiny', 'base', 'small', 'medium', 'large-v3'
model_name = 'medium'

# 语言
language = 'zh'

# 初始提示，可选
initial_prompt = '请用中文'

# 采样温度
temperature = 0.0

# 输出文件后缀
suffix = '信号与人生'

# 字幕文件路径
output_subtitle_path = f"./output-{suffix}.srt"

# 模型缓存目录
cache_dir = './'

### 运行语音识别


In [6]:
# 构建解码选项
decode_options = {
    "language": language,
    "initial_prompt": initial_prompt,
    "temperature": temperature
}

# 运行 ASR
speech_recognition(
    model_name=model_name,
    input_audio=input_audio_array,
    output_subtitle_path=output_subtitle_path,
    decode_options=decode_options,
    cache_dir=cache_dir
)

100%|███████████████████████████████████████████████████████████| 104500/104500 [01:10<00:00, 1479.80frames/s]

字幕已保存到 ./output-信号与人生.srt


### 检查结果


In [7]:
# 读取并打印字幕内容
with open(output_subtitle_path, 'r', encoding='utf-8') as file:
    content = file.read()
print(content)

1
00:00:00,000 --> 00:00:04,000
每次说 学问是做出来的

2
00:00:06,000 --> 00:00:08,000
什么意思

3
00:00:08,000 --> 00:00:12,000
要做 才会获得学问

4
00:00:13,000 --> 00:00:16,000
你如果每天光是坐在那里听

5
00:00:17,000 --> 00:00:20,000
学问很可能是左耳近右耳出的

6
00:00:21,000 --> 00:00:23,000
你光是坐在那儿读

7
00:00:23,000 --> 00:00:26,000
学问可能从眼睛进入脑海之后就忘掉了

8
00:00:26,000 --> 00:00:29,000
如何能够学问在脑海里面

9
00:00:31,000 --> 00:00:33,000
真的变成你自己学问

10
00:00:33,000 --> 00:00:35,000
就是要做

11
00:00:36,000 --> 00:00:39,000
可能有很多同学有这个经验

12
00:00:39,000 --> 00:00:41,000
你如果去修某一门课

13
00:00:41,000 --> 00:00:44,000
或者做某一个实验

14
00:00:44,000 --> 00:00:47,000
在期末就是要教一个final project

15
00:00:48,000 --> 00:00:50,000
那个final project就是要你把

16
00:00:51,000 --> 00:00:53,000
学到的很多东西

17
00:00:53,000 --> 00:00:56,000
最后整合在你的final project里面

18
00:00:56,000 --> 00:00:58,000
最后做出来的时候

19
00:00:58,000 --> 00:01:00,000
就是把它们都整合了

20
00:01:00,000 --> 00:01:02,000
当你学期结束

21
00:01:02,000 --> 00:01:04,000
真的把final project做完的时候

22
00:01:04,000 --> 00:01:05,000

## 第 3 部分 - 处理自动语音识别的结果

### 提取字幕文本

In [8]:
def extract_and_save_text(srt_filename, output_filename):
    # 读取 SRT 文件
    with open(srt_filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # 去除时间戳和索引
    pure_text = re.sub(r'\d+\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n', '', content)
    pure_text = re.sub(r'\n\n+', '\n', pure_text)

    # 保存纯文本
    with open(output_filename, 'w', encoding='utf-8') as output_file:
        output_file.write(pure_text)

    print(f'提取的文本已保存到 {output_filename}')

    return pure_text

### 拆分文本



In [9]:
def chunk_text(text, max_length):
    return textwrap.wrap(text, max_length)

### 执行文本处理



In [10]:
# 文本块长度
chunk_length = 512

# 是否转换为繁体中文
convert_to_traditional = False

# 提取文本并拆分
pure_text = extract_and_save_text(
    srt_filename=output_subtitle_path,
    output_filename=f"./output-{suffix}.txt",
)

chunks = chunk_text(text=pure_text, max_length=chunk_length)

提取的文本已保存到 ./output-信号与人生.txt


## 第 4 部分 - 文本摘要

### 设置 OpenAI API

首先，需要设置 OpenAI API 密钥，这里使用的是阿里云的大模型，你可以通过[《00. 阿里大模型API获取步骤》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/00.%20阿里大模型API获取步骤.md)获取 API 密钥。

如果需要使用其他平台，请参考对应的开发文档后对应修改 base_url。

In [11]:
# 设置 OpenAI API 密钥
openai_api_key = '替换成你的 API 密钥'

# 构建 OpenAI 客户端
client = OpenAI(
    api_key=openai_api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", # 这里使用的是阿里云的大模型，如果需要使用其他平台，请参考对应的开发文档后对应修改。如果使用 GPT 的 API，删除这行就可以直接运行。
)


### 设置参数

默认使用 qwen-turbo，其他模型可以参阅[模型广场 -- 阿里云百炼](https://bailian.console.aliyun.com/?spm=5176.29619931.J__Z58Z6CX7MY__Ll8p1ZOR.1.4d1d59fcWwSqvr#/model-market)，点击对应模型的`查看详情`。

![image-20240924091151684](../Guide/assets/image-20240924091151684.png)

在界面可以左上角看到对应的英文名称，复制它，然后替换 `model_name`。

![image-20240924091414350](../Guide/assets/image-20240924091414350.png)

你可以随意更换为你想要的模型，不过可能要先申请使用（通过大概要几个小时，会有短信提示）。

In [12]:
# 模型名称
model_name = 'qwen-turbo'

# 控制响应的随机性
temperature = 0.0

# 控制多样性
top_p = 1.0

# 最大生成标记数
max_tokens = 512

### 定义摘要函数


In [13]:
def summarization(client, summarization_prompt, model_name="qwen-turbo", temperature=0.0, top_p=1.0, max_tokens=512):
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": summarization_prompt}],
        model=model_name,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

### 这里演示两种摘要方式

分别对应于 `MapReduce` 和 `Refine`，我会在之后具体讲解它们的区别，你可以通过接下来的代码来感受二者直观的区别。

#### 拆分为多段进行摘要（Multi-Stage Summarization）- MapReduce

![image.png](../Guide/assets/image-20240924092040340.png)

1. 将长文本分成多个较小的部分，并分别获取每个小段落的摘要

In [14]:
# 定义摘要提示模板
summarization_prompt_template = "用 300 个字以内写出这段视频文本的摘要，其中包括要点和所有重要细节：<text>"

# 对每个文本块生成摘要
paragraph_summaries = []
for index, chunk in enumerate(chunks):
    print(f"\n========== 正在生成第 {index + 1} 段摘要 ==========\n")
    print(f"原始文本 (第 {index + 1} 段):\n{chunk}\n")
    
    # 构建摘要提示
    summarization_prompt = summarization_prompt_template.replace("<text>", chunk)
    
    # 调用摘要函数
    summary = summarization(
        client=client,
        summarization_prompt=summarization_prompt,
        model_name=model_name,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )
    
    # 打印生成的摘要
    print(f"生成的摘要 (第 {index + 1} 段):\n{summary}\n")
    
    # 将生成的摘要保存到列表
    paragraph_summaries.append(summary)


========== 正在生成第 1 段摘要 ==========

原始文本 (第 1 段):
每次说 学问是做出来的 什么意思 要做 才会获得学问 你如果每天光是坐在那里听 学问很可能是左耳近右耳出的 你光是坐在那儿读 学问可能从眼睛进入脑海之后就忘掉了 如何能够学问在脑海里面 真的变成你自己学问 就是要做 可能有很多同学有这个经验 你如果去修某一门课 或者做某一个实验 在期末就是要教一个final project 那个final project就是要你把 学到的很多东西 最后整合在你的final project里面 最后做出来的时候 就是把它们都整合了 当你学期结束 真的把final project做完的时候 你会忽然发现 我真的学到很多东西 那就是做出来的学问 也许可以举另外一个例子 就是你如果学了某一些 很复杂的演算法或者什么 好像觉得那些不见得在你的脑海里 可是后来老师出了个习题 那个习题叫你写一个很大的程式 要把所有东西都包进去 当你把这个程式写完的时候 你会发现 你忽然把演算法 所有东西都弄通了 那就是学问是做出来的 所以我们永远要记得 尽量多动手多做 在动手跟做的过程之中 学问才可以变成是自己的 同样的情形就是说 很多时候这样动手或者做的 表现或者成绩 没有一个成绩单上的数字

生成的摘要 (第 1 段):
这段视频强调“学问是做出来的”这一理念。主要观点是，仅仅听或阅读知识不足以使学问真正成为个人的知识。只有通过实践和动手操作，学问才能内化为个人的能力。

首先，视频提到，如果只听不实践，学到的知识很可能只是短暂记忆；同样，如果只读不思辨，知识也可能在大脑中稍纵即逝。因此，要让知识转化为真正的学问，必须付诸实践。

其次，视频举例说明学习过程中实践的重要性。例如，在课程期末进行的最终项目，要求学生整合并应用整学期所学内容。当学生实际完成最终项目时，他们会惊奇地发现，自己真的学到了很多。

再如，学习复杂算法时，理论知识可能难以在大脑中留下深刻印象。然而，一旦遇到需要将所有知识点应用于实际编程的题目时，学生不仅会发现自己的理论知识得到了运用，而且对这些知识的理解也会更加深入。

视频反复强调“动手”的重要性。只有通过实践、操作、解决问题，知识才能在脑海中真正扎根，并成为个人能力的一部分。此外，这种学习方式下，个人表现和成就往往无法简单通过成绩单上

2. 在分别获取每个小段落的摘要后，处理这些摘要以生成最终的摘要。

In [15]:
# 合并段落摘要
collected_summaries = "\n".join(paragraph_summaries)

# 定义最终摘要提示模板
final_summarization_prompt = "在 500 字以内写出以下文字的简洁摘要：<text>"
final_summarization_prompt = final_summarization_prompt.replace("<text>", collected_summaries)

# 生成最终摘要
final_summary = summarization(
    client=client,
    summarization_prompt=final_summarization_prompt,
    model_name=model_name,
    temperature=temperature,
    top_p=top_p,
    max_tokens=max_tokens
)

print(final_summary)

这段视频强调“学问是做出来的”这一理念，指出仅仅听讲或阅读知识不足以使学问真正成为个人的知识。它主张实践和动手操作是将知识转化为个人能力的关键。视频通过举例说明，无论是课程中末期的项目、整合并应用整个学期内容的任务，还是在复杂的算法学习中，实际操作和问题解决过程对理解和记忆知识具有重要作用。视频反复强调“动手”和实践的重要性，认为这种学习方式能将知识内化，形成个人技能，并实现更深层次的学习和理解。它反对那种可能忽视过程与实践的学习方法，并主张在学术之外寻找学习机会，比如社交活动、志愿者服务、社团活动等，以发展软实力并促进个人成长。视频以个人经历为例，解释了电机系工程师在不同生活阶段的发展轨迹，强调实力、努力、机遇与自我技能四个因素对个人成长的影响，并鼓励个体设定并追求长程目标，以实现全面且平衡的个人发展。


#### 精炼方法（the method of Refinement) - Refine

Refinement 就是把每次的文本和之前的摘要结合起来丢给大模型，类似于迭代：

![Refinement](../Guide/assets/image-20240924092753352.png)

步骤（Pipeline）如下：
- 第1步：从一小部分数据开始，运行prompt生成初始输出。
- 第2步：对后续每个文档，将前一个输出与新文档结合输入。
- 第3步：LLM 根据新文档中的信息精炼输出。
- 第4步：此过程持续迭代，直到处理完所有文档。

对应的核心代码：

In [16]:
# 定义初始摘要提示模板
summarization_prompt_template = "用 300 个字以内写出这段视频文本的摘要，其中包括要点和所有重要细节:<text>"

# 定义精炼摘要提示模板
summarization_prompt_refinement_template = "请在 500 字以内，结合原先的摘要和新的内容，提供简洁的摘要:<text>"

# 初始化保存摘要的列表
refined_summaries = []

# 对文本块逐步进行精炼摘要，并打印中间过程
for index, chunk in enumerate(chunks):
    if index == 0:
        # 第一步：对第一段文本生成初始摘要
        print(f"\n========== 正在生成第 {index + 1} 段的初始摘要 ==========\n")
        print(f"原始文本 (第 {index + 1} 段):\n{chunk}\n")
        
        # 构建初始摘要提示
        summarization_prompt = summarization_prompt_template.replace("<text>", chunk)
        
        # 调用摘要函数生成第一个摘要
        first_summary = summarization(
            client=client,
            summarization_prompt=summarization_prompt,
            model_name=model_name,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens
        )
        
        # 打印生成的初始摘要
        print(f"生成的摘要 (第 {index + 1} 段):\n{first_summary}\n")
        
        # 保存生成的摘要
        refined_summaries.append(first_summary)

    else:
        # 后续步骤：结合前一个摘要与当前段落进行精炼
        print(f"\n========== 正在生成第 {index + 1} 段的精炼摘要 ==========\n")
        print(f"原始文本 (第 {index + 1} 段):\n{chunk}\n")
        
        # 构建精炼摘要的输入文本，将前一个摘要与当前段落内容结合
        chunk_with_previous_summary = f"前 {index} 段的摘要: {refined_summaries[-1]}\n第 {index + 1} 段的内容: {chunk}"
        
        # 构建精炼摘要提示
        summarization_prompt = summarization_prompt_refinement_template.replace("<text>", chunk_with_previous_summary)
        
        # 调用摘要函数生成精炼摘要
        refined_summary = summarization(
            client=client,
            summarization_prompt=summarization_prompt,
            model_name=model_name,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens
        )
        
        # 打印生成的精炼摘要
        print(f"生成的摘要 (第 {index + 1} 段):\n{refined_summary}\n")
        
        # 保存生成的精炼摘要
        refined_summaries.append(refined_summary)

# 最终的精炼摘要结果就是 refined_summaries 列表的最后一个元素
final_refined_summary = refined_summaries[-1]

print("\n========== 最终精炼摘要结果 ==========\n")
print(final_refined_summary)


========== 正在生成第 1 段的初始摘要 ==========

原始文本 (第 1 段):
每次说 学问是做出来的 什么意思 要做 才会获得学问 你如果每天光是坐在那里听 学问很可能是左耳近右耳出的 你光是坐在那儿读 学问可能从眼睛进入脑海之后就忘掉了 如何能够学问在脑海里面 真的变成你自己学问 就是要做 可能有很多同学有这个经验 你如果去修某一门课 或者做某一个实验 在期末就是要教一个final project 那个final project就是要你把 学到的很多东西 最后整合在你的final project里面 最后做出来的时候 就是把它们都整合了 当你学期结束 真的把final project做完的时候 你会忽然发现 我真的学到很多东西 那就是做出来的学问 也许可以举另外一个例子 就是你如果学了某一些 很复杂的演算法或者什么 好像觉得那些不见得在你的脑海里 可是后来老师出了个习题 那个习题叫你写一个很大的程式 要把所有东西都包进去 当你把这个程式写完的时候 你会发现 你忽然把演算法 所有东西都弄通了 那就是学问是做出来的 所以我们永远要记得 尽量多动手多做 在动手跟做的过程之中 学问才可以变成是自己的 同样的情形就是说 很多时候这样动手或者做的 表现或者成绩 没有一个成绩单上的数字

生成的摘要 (第 1 段):
这段视频讲述了"学问是做出来的"这一理念。关键点包括：1）学问不只是通过听讲或阅读获取的，而是通过实际操作和实践来获得的。2）长时间被动接收信息可能导致知识的遗忘，而参与实践活动能帮助将知识内化为自己的学问。3）具体例子如，通过完成课程项目或解决复杂问题，学习者能将所学的知识整合并真正掌握，从而获得深入的理解。4）实践过程中，不仅知识得以巩固，技能也得到提升，这些是成绩单上无法体现的软技能。最终，强调行动的重要性，即通过动手操作，知识才能转化为个人的学问。


========== 正在生成第 2 段的精炼摘要 ==========

原始文本 (第 2 段):
使得很多人觉得那不重要 很多人甚至觉得 这门课要做final project 我就不修了太累了 或者说那门课需要 怎么样太累我就不要做了 而不知道 其实那个才是让你做的机会 然后可以学到最多 也就是说虽然很可能 那么辛苦的做很多事 没有让你获得什么具体成绩 对你的overfit